In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
#For reproductivility
torch.manual_seed(1)

In [3]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [5]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [6]:
print('e^1 equals : ', torch.exp(torch.FloatTensor([1])))

e^1 equals :  tensor([2.7183])


In [7]:
w = torch.zeros((2, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

In [8]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(w) + b)))

In [9]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [10]:
print('1/(1+e^{-1}) equals :', torch.sigmoid(torch.FloatTensor([1])))

1/(1+e^{-1}) equals : tensor([0.7311])


In [11]:
hypothesis = torch.sigmoid(x_train.matmul(w) + b)

In [12]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


In [13]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [14]:
-(y_train[0] * torch.log(hypothesis[0]) + (1 - y_train[0]) * torch.log(1 - hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward>)

In [15]:
losses = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis))

In [16]:
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)


In [17]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [18]:
#지금까지 적은 수식들을 한번에 해결 가능
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

In [19]:
#전체 트레이닝 절차

In [20]:
#모델 초기화
w = torch.zeros((2, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

#optimizer 설정
optimizer = optim.SGD([w, b], lr = 1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    #Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(w) + b) #or .nn or #
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    #cost로 W(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epochs {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epochs    0/1000 Cost: 0.693147
Epochs  100/1000 Cost: 0.134722
Epochs  200/1000 Cost: 0.080643
Epochs  300/1000 Cost: 0.057900
Epochs  400/1000 Cost: 0.045300
Epochs  500/1000 Cost: 0.037261
Epochs  600/1000 Cost: 0.031673
Epochs  700/1000 Cost: 0.027556
Epochs  800/1000 Cost: 0.024394
Epochs  900/1000 Cost: 0.021888
Epochs 1000/1000 Cost: 0.019852


In [21]:
#Evaluation
hypothesis = torch.sigmoid(x_train.matmul(w) + b)
#hypothesis H(xi=1)일 확률이 출력된다.
print(hypothesis[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward>)


In [22]:
#여기에 대해 확률을 구했으니, 0인지, 1인지 찍어야 한다.
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [23]:
#결과 확인
print(prediction[:5])
print(y_train[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [24]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


In [25]:
#실전에서는 다음과 같이 구현
#nn.Module이라는 추상클래스를 상속받아, 자신만의 class 제작
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        #self.linear 은 w와 b 두개를 가지고 있다. w는 8*1, b는 1의 shape를 갖는다.
        self.linear = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [26]:
model = BinaryClassifier()